In [67]:
import numpy as np

#Définition d'un line world sous la forme d'un MDP

In [29]:
S = [0, 1, 2 ,3, 4]
A = [0, 1] # left right
R = [-1, 0, 1]
T = [0, 4] # ensemble des états terminaux
p = np.zeros((len(S), len(A), len(S), len(R))) # S, A, S_p, R
# si t'es à l'état S, que tu fais l'action A, quelle est la proba
# de se retrouver à l'état S+1 avec le Reward R

In [17]:
for s in range(len(S)):
  for a in range(len(A)):
    for s_p in range(len(S)):
      for r in range(len(R)):
        if s_p == s + 1 and a == 1 and R[r] == 0 and s in [1, 2]:
          p[s, a, s_p, r] = 1.0
        if s_p == s - 1 and a == 0 and R[r] == 0 and s in [2, 3]:
          p[s, a, s_p, r] = 1.0
p[3, 1, 4, 2] = 1.0
p[1, 0, 0, 0] = 1.0

# Exercice

Implémentez une fonction prenant en paramètre une policy (un tableau numpy)
et renvoyant la value de cette policy (un tableau numpy) pour chaque état du line world

In [4]:
def policy_evaluation(policy : np.ndarray, V: np.ndarray = None) -> np.ndarray:
  theta = 0.00001
  V = np.random.random(len(S))
  for s in T:
    V[s] = 0.0
  gamma = 0.999
  while True:
    delta = 0
    for s in range(len(S)):
      v = V[s]
      total = 0
      for a in range(len(A)):
        pi_s_a = policy[s, a]
        for s_p in range(len(S)):
          for r in range(len(R)):
            total += pi_s_a * p[s, a, s_p, r] * (R[r] + gamma * V[s_p])
      V[s] = total
      delta = np.maximum(delta, np.abs(v - V[s]))
    if delta < theta:
      return V

Définissez une policy jouant tout le temps à droite.
Affichez la value de cette policy obtenue grâce à l'algorithme policy evaluation

In [5]:
pi_right = np.array([[0.0, 1.0] for _ in range(len(S))])
print(policy_evaluation(pi_right))

[0.       0.998001 0.999    1.       0.      ]


Définissez une policy jouant tout le temps à gauche. Affichez la value de cette policy obtenue grâce à l'algorithme policy evaluation

In [6]:
pi_left = np.array([[1.0, 0.0] for _ in range(len(S))])
print(policy_evaluation(pi_left))

[ 0.       -1.       -0.999    -0.998001  0.      ]


Définissez une policy uniformément aléatoire (50% de chance d'aller à gauche et 50% de chances d'aller à droite). Affichez la value de cette policy obtenue grâce à l'algorithme policy evaluation

In [7]:
pi_random = np.array([[0.5, 0.5] for _ in range(len(S))])
print(policy_evaluation(pi_random))

[ 0.00000000e+00 -4.99994087e-01  5.90749822e-06  5.00002951e-01
  0.00000000e+00]


Définissez une policy uniformément aléatoire (15% de chance d'aller à gauche et 85% de chances d'aller à droite). Affichez la value de cette policy obtenue grâce à l'algorithme policy evaluation

In [8]:
pi_weird_random = np.array([[0.15, 0.85] for _ in range(len(S))])
print(policy_evaluation(pi_weird_random))

[0.         0.64651346 0.93801509 0.99056156 0.        ]


In [9]:
def naive_ineficient_policy_iteration():
  theta = 0.000001
  gamma = 0.999

  V = np.random.random(len(S))
  for s in T:
    V[s] = 0.0

  Pi = np.random.choice(A, len(S))

  while True:
    # Policy Evaluation
    while True:
      delta = 0.0

      for s in S:
        v = V[s]

        total = 0.0
        for s_p in S:
          for r_index in range(len(R)):
            total += p[s, Pi[s], s_p, r_index] * (R[r_index] + gamma * V[s_p])

        V[s] = total

        delta = max(delta, abs(v - V[s]))

      if delta < theta:
        break

    # Policy Improvement
    policy_stable = True

    for s in S:
      old_action = Pi[s]

      best_a = None
      best_action_score = -9999999

      for a in A:
        total = 0.0
        for s_p in S:
          for r_index in range(len(R)):
            total += p[s, a, s_p, r_index] * (R[r_index] + gamma * V[s_p])

        if best_a is None or total >= best_action_score:
          best_a = a
          best_action_score = total

      Pi[s] = best_a
      if Pi[s] != old_action:
        policy_stable = False

    if policy_stable:
      return Pi

In [10]:
naive_ineficient_policy_iteration()

array([1, 1, 1, 1, 1])

# Définissez une variante d'un GRID WORLD sous la forme d'un MDP et évaluez différentes stratégies sur cette environnement.

Le grid world est une grille de 5x5 cases (5 lignes de 5 colonnes) sur laquelle l'agent peut évoluer, il commence généralement sur la première ligne, première colonne. L'agent possède 4 actions possibles (gauche, droite, haut, bas). Si jamais l'agent atteint la dernière case de la première ligne => état terminal avec reward de -3. Si jamais l'agent atteint la dernière case de la dernière ligne => état terminal avec reward de 1. Si l'agent essaye de se déplacer en dehors des bords de la grille => état terminal avec score de -1.

In [3]:
#S = [[0, 1, 2, 3, 4, 5, 6], 
#     [0, 1, 2, 3, 4, 5, 6],
#     [0, 1, 2, 3, 4, 5, 6],
#     [0, 1, 2, 3, 4, 5, 6],
#     [0, 1, 2, 3, 4, 5, 6],
#     [0, 1, 2, 3, 4, 5, 6]] # premiere ligne et derniere lignes outside of game, col 0 et col 6 outside of game

#S = [ 0,  1,  2,  3,  4,  5,  6, 
#      7,  8,  9, 10, 11, 12, 13,
#     14, 15, 16, 17, 18, 19, 20,
#     21, 22, 23, 24, 25, 26, 27,
#     28, 29, 30, 31, 32, 33, 34,
#     35, 36, 37, 38, 39, 40, 41]

import numpy as np

S = [(i, j) for i in range(6) for j in range(7)]
A = [0, 1, 2, 3] # left right up down
R = [-3, -1, 0, 1] 
T = [(1,5), (4,5), (0,0), (0,1), (0,2), (0,3), (0,4), (0,5), (0,6), (5,0), (5,1), (5,2), (5,3), (5,4), (5,5), (5,6), (1,0), (2,0), (3,0), (4,0), (1,6), (2,6), (3,6), (4,6)]

p = np.zeros((6, 7, 4, 6, 7, 4)) # S, A, S_p, R

In [4]:
for s_line in range(6):
  for s_column in range(7):
    for a in range(len(A)):
      for s_p_line in range(6):
        for s_p_column in range(7):
          for r in range(len(R)):
            # si on est sur la ligne 1 (pas 0) et qu'on monte 
            if s_line == 1 and 5 > s_column > 0 and a == 2 and s_p_line == s_line - 1 and s_p_column == s_column and R[r] == -1 :
              p[s_line][s_column][a][s_p_line][s_p_column][r] = 1.0
            # si on est sur la colonne 1 (pas 0) et qu'on va à gauche
            if 0 < s_line < 5 and s_column == 1 and a == 0 and s_p_line == s_line and s_p_column == 0 and R[r] == -1 :
              p[s_line][s_column][a][s_p_line][s_p_column][r] = 1.0
            # si on est sur la colonne 5 et qu'on va à droite
            if 0 < s_line < 5 and s_column == 5 and a == 1 and s_p_line == s_line and s_p_column == 6 and R[r] == -1 :
              p[s_line][s_column][a][s_p_line][s_p_column][r] = 1.0
            # si on est sur la derniere ligne et qu'on descend
            if s_line == 4 and 0 < s_column < 5 and a == 3 and s_p_line == 5 and s_p_column == s_column and R[r] == -1 :
              p[s_line][s_column][a][s_p_line][s_p_column][r] = 1.0                   
                
            # si on est sur la ligne 1 (pas 0) et qu'on va à droite 
            if s_line == 1 and 4 > s_column > 0 and a == 1 and s_p_line == s_line and s_p_column == s_column + 1 and R[r] == 0 :
              p[s_line][s_column][a][s_p_line][s_p_column][r] = 1.0
            # si on est sur la ligne 1 (pas 0) et qu'on va à gauche 
            if s_line == 1 and 5 > s_column > 1 and a == 0 and s_p_line == s_line and s_p_column == s_column - 1 and R[r] == 0 :
              p[s_line][s_column][a][s_p_line][s_p_column][r] = 1.0  
            # si on est sur la ligne 1 (pas 0) et qu'on descend 
            if s_line == 1 and 5 > s_column > 0 and a == 3 and s_p_line == s_line + 1 and s_p_column == s_column and R[r] == 0 :
              p[s_line][s_column][a][s_p_line][s_p_column][r] = 1.0  

            # si on est sur la ligne 2 et qu'on va à droite 
            if s_line == 2 and 5 > s_column > 0 and a == 1 and s_p_line == s_line and s_p_column == s_column + 1 and R[r] == 0 :
              p[s_line][s_column][a][s_p_line][s_p_column][r] = 1.0
            # si on est sur la ligne 2 et qu'on va à gauche 
            if s_line == 2 and 6 > s_column > 2 and a == 0 and s_p_line == s_line and s_p_column == s_column - 1 and R[r] == 0 :
              p[s_line][s_column][a][s_p_line][s_p_column][r] = 1.0  
            # si on est sur la ligne 2 et qu'on descend 
            if s_line == 2 and 6 > s_column > 0 and a == 3 and s_p_line == s_line + 1 and s_p_column == s_column and R[r] == 0 :
              p[s_line][s_column][a][s_p_line][s_p_column][r] = 1.0 
            # si on est sur la ligne 2 et qu'on monte 
            if s_line == 2 and 5 > s_column > 0 and a == 2 and s_p_line == s_line - 1 and s_p_column == s_column and R[r] == 0 :
              p[s_line][s_column][a][s_p_line][s_p_column][r] = 1.0 
              
            # si on est sur la ligne 3 et qu'on va à droite 
            if s_line == 3 and 5 > s_column > 0 and a == 1 and s_p_line == s_line and s_p_column == s_column + 1 and R[r] == 0 :
              p[s_line][s_column][a][s_p_line][s_p_column][r] = 1.0
            # si on est sur la ligne 3 et qu'on va à gauche 
            if s_line == 3 and 6 > s_column > 2 and a == 0 and s_p_line == s_line and s_p_column == s_column - 1 and R[r] == 0 :
              p[s_line][s_column][a][s_p_line][s_p_column][r] = 1.0  
            # si on est sur la ligne 3 et qu'on descend 
            if s_line == 3 and 5 > s_column > 0 and a == 3 and s_p_line == s_line + 1 and s_p_column == s_column and R[r] == 0 :
              p[s_line][s_column][a][s_p_line][s_p_column][r] = 1.0
            # si on est sur la ligne 3 et qu'on monte 
            if s_line == 3 and 6 > s_column > 0 and a == 2 and s_p_line == s_line - 1 and s_p_column == s_column and R[r] == 0 :
              p[s_line][s_column][a][s_p_line][s_p_column][r] = 1.0               
          
            # si on est sur la ligne 4 et qu'on va à droite 
            if s_line == 4 and 4 > s_column > 0 and a == 1 and s_p_line == s_line and s_p_column == s_column + 1 and R[r] == 0 :
              p[s_line][s_column][a][s_p_line][s_p_column][r] = 1.0
            # si on est sur la ligne 4 et qu'on va à gauche 
            if s_line == 4 and 5 > s_column > 2 and a == 0 and s_p_line == s_line and s_p_column == s_column - 1 and R[r] == 0 :
              p[s_line][s_column][a][s_p_line][s_p_column][r] = 1.0  
            # si on est sur la ligne 4 et qu'on monte 
            if s_line == 3 and 5 > s_column > 0 and a == 2 and s_p_line == s_line - 1 and s_p_column == s_column and R[r] == 0 :
              p[s_line][s_column][a][s_p_line][s_p_column][r] = 1.0 
              
# cas avec reward spéciaux
p[1][4][1][1][5][0] = 1.0
p[2][5][2][1][5][0] = 1.0

p[3][5][3][4][5][3] = 1.0
p[4][4][1][4][5][3] = 1.0
            

In [80]:
# affichage de p pour comprendre avec quoi on travaille
for s_line in range(6):
  for s_column in range(7):
    for a in range(len(A)):
      for s_p_line in range(6):
        for s_p_column in range(7):
          for r in range(len(R)):
            if p[s_line][s_column][a][s_p_line][s_p_column][r] == 1:
              print("p[{}][{}][{}][{}][{}][{}] = {}".format(s_line, s_column, a, s_p_line, s_p_column, r, p[s_line][s_column][a][s_p_line][s_p_column][r]))

p[1][1][0][1][0][1] = 1.0
p[1][1][1][1][2][2] = 1.0
p[1][1][2][0][1][1] = 1.0
p[1][1][3][2][1][2] = 1.0
p[1][2][0][1][1][2] = 1.0
p[1][2][1][1][3][2] = 1.0
p[1][2][2][0][2][1] = 1.0
p[1][2][3][2][2][2] = 1.0
p[1][3][0][1][2][2] = 1.0
p[1][3][1][1][4][2] = 1.0
p[1][3][2][0][3][1] = 1.0
p[1][3][3][2][3][2] = 1.0
p[1][4][0][1][3][2] = 1.0
p[1][4][1][1][5][0] = 1.0
p[1][4][2][0][4][1] = 1.0
p[1][4][3][2][4][2] = 1.0
p[1][5][1][1][6][1] = 1.0
p[2][1][0][2][0][1] = 1.0
p[2][1][1][2][2][2] = 1.0
p[2][1][2][1][1][2] = 1.0
p[2][1][3][3][1][2] = 1.0
p[2][2][1][2][3][2] = 1.0
p[2][2][2][1][2][2] = 1.0
p[2][2][3][3][2][2] = 1.0
p[2][3][0][2][2][2] = 1.0
p[2][3][1][2][4][2] = 1.0
p[2][3][2][1][3][2] = 1.0
p[2][3][3][3][3][2] = 1.0
p[2][4][0][2][3][2] = 1.0
p[2][4][1][2][5][2] = 1.0
p[2][4][2][1][4][2] = 1.0
p[2][4][3][3][4][2] = 1.0
p[2][5][0][2][4][2] = 1.0
p[2][5][1][2][6][1] = 1.0
p[2][5][2][1][5][0] = 1.0
p[2][5][3][3][5][2] = 1.0
p[3][1][0][3][0][1] = 1.0
p[3][1][1][3][2][2] = 1.0
p[3][1][2][2

# Proposez plusieurs stratégies et évaluez les à l'aide de policy évaluation.

In [ ]:
#TODO

# Bonus : Implémentez policy itération et value itération (dans les slides que nous n'avons pas encore vu) et obtenez à l'aide de ces derniers pi* pour le Line World et le Grid World

In [ ]:
#TODO

# Naive Policy iteration implementation in order to find a Pi*

In [24]:
def naive_policy_iteration():
  theta = 0.00001
  gamma = 0.999

  V = np.random.random(len(S))
  for s in T:
    V[s] = 0.0
  Pi = np.random.choice(A, len(S), True)

  while True:
    # Policy evaluation
    while True:
      delta = 0
      for s in range(len(S)):
        v = V[s]
        total = 0
        for s_p in range(len(S)):
          for r in range(len(R)):
            total += p[s, Pi[s], s_p, r] * (R[r] + 0.999 * V[s_p])
        V[s] = total
        delta = np.maximum(delta, np.abs(v - V[s]))
      if delta < theta:
        break

    # Policy improvement
    policy_stable = True

    for s in S:
      if s in T:
        continue

      old_action = Pi[s]

      # Compute Arg Max a
      argmax_a = None
      max_a = -999999999

      for a in A:
        total = 0.0

        for s_p in S:
          for r_index in range(len(R)):
            total += p[s, a, s_p, r_index] * (R[r_index] + gamma * V[s_p])

        if argmax_a is None or total >= max_a:
          argmax_a = a
          max_a = total

      Pi[s] = argmax_a

      if old_action != Pi[s]:
        policy_stable = False


    if policy_stable:
      break

  return Pi


In [26]:
tab = naive_policy_iteration()

# Grid World avec vecteurs au lieu de matrices

In [1]:
#S = [ 0,  1,  2,  3,  4,  5,  6,   LIGNE 1
#      7,  8,  9, 10, 11, 12, 13,   LIGNE 2
#     14, 15, 16, 17, 18, 19, 20,   LIGNE 3
#     21, 22, 23, 24, 25, 26, 27,   LIGNE 4
#     28, 29, 30, 31, 32, 33, 34,   LIGNE 5
#     35, 36, 37, 38, 39, 40, 41,   LIGNE 6
#     42, 43, 44, 45, 46, 47, 48]   LIGNE 7     A = [0, 1, 2, 3] # left right up down

import numpy as np

S = [i for i in range(49)]
A = [0, 1, 2, 3] # left right up down
R = [-3, -1, 0, 1] 
T = [0, 1, 2, 3, 4, 5, 6, 7, 12, 13, 14, 20, 21, 27, 28, 34, 35, 40, 41, 42, 43, 44, 45, 46, 47, 48]

p = np.zeros((len(S), len(A), len(S), len(R))) # S, A, S_p, R

In [5]:
for s in range(len(S)):
  for a in range(len(A)):
    for s_p in range(len(S)):
      for r in range(len(R)):
      

        # actions terminales :
        # si on monte depuis la premiere ligne :
        if 7 < s < 12 and a == 2 and s_p == s - 7 and R[r] == -1 :
          p[s, a, s_p, r] = 1.0
        # si on descend depuis la derniere ligne :
        if 35 < s < 40 and a == 3 and s_p == s + 7 and R[r] == -1 :
          p[s, a, s_p, r] = 1.0
        # si on va à gauche depuis la première colonne :
        if s % 7 == 1 and 7 < s < 37 and a == 0 and s_p == s - 1 and R[r] == -1 :
          p[s, a, s_p, r] = 1.0
        # si on va à droite depuis la dernière colonne :
        if s % 7 == 5 and 7 < s < 37 and a == 1 and s_p == s + 1 and R[r] == -1 :
          p[s, a, s_p, r] = 1.0
            
        # actions banales :    
        # si on est sur la premiere ligne:
        # si on descend
        if 7 < s < 12 and a == 3 and s_p == s + 7 and R[r] == 0 :
          p[s, a, s_p, r] = 1.0
        # si on va à gauche
        if 8 < s < 12 and a == 0 and s_p == s - 1 and R[r] == 0 :
          p[s, a, s_p, r] = 1.0
        # si on va à droite
        if 7 < s < 11 and a == 1 and s_p == s + 1 and R[r] == 0 :
          p[s, a, s_p, r] = 1.0


        # si on est sur la deuxième ligne:
        # si on monte
        if 14 < s < 19 and a == 2 and s_p == s - 7 and R[r] == 0 :
          p[s, a, s_p, r] = 1.0
        # si on descend
        if 14 < s < 20 and a == 3 and s_p == s + 7 and R[r] == 0 :
          p[s, a, s_p, r] = 1.0
        # si on va à droite
        if 14 < s < 19 and a == 1 and s_p == s + 1 and R[r] == 0 :
          p[s, a, s_p, r] = 1.0
        # si on va à gauche
        if 15 < s < 20 and a == 0 and s_p == s - 1 and R[r] == 0 :
          p[s, a, s_p, r] = 1.0

#S = [ 0,  1,  2,  3,  4,  5,  6,   LIGNE 0
#      7,  8,  9, 10, 11, 12, 13,   LIGNE 1
#     14, 15, 16, 17, 18, 19, 20,   LIGNE 2
#     21, 22, 23, 24, 25, 26, 27,   LIGNE 3
#     28, 29, 30, 31, 32, 33, 34,   LIGNE 4
#     35, 36, 37, 38, 39, 40, 41,   LIGNE 5
#     42, 43, 44, 45, 46, 47, 48]   LIGNE 6     A = [0, 1, 2, 3] # left right up down        
        
        # si on est sur la troisième ligne:
        # si on monte
        if 21 < s < 27 and a == 2 and s_p == s - 7 and R[r] == 0 :
          p[s, a, s_p, r] = 1.0
        # si on descend
        if 21 < s < 27 and a == 3 and s_p == s + 7 and R[r] == 0 :
          p[s, a, s_p, r] = 1.0
        # si on va à gauche
        if 22 < s < 27 and a == 0 and s_p == s - 1 and R[r] == 0 :
          p[s, a, s_p, r] = 1.0
        # si on va à droite
        if 21 < s < 26 and a == 1 and s_p == s + 1 and R[r] == 0 :
          p[s, a, s_p, r] = 1.0
          
        
        # si on est sur la quatrième ligne:
        # si on monte
        if 28 < s < 34 and a == 2 and s_p == s - 7 and R[r] == 0 :
          p[s, a, s_p, r] = 1.0
        # si on descend
        if 28 < s < 33 and a == 3 and s_p == s + 7 and R[r] == 0 :
          p[s, a, s_p, r] = 1.0 
        # si on va à gauche
        if 29 < s < 34 and a == 0 and s_p == s - 1 and R[r] == 0 :
          p[s, a, s_p, r] = 1.0 
        # si on va à droite
        if 28 < s < 33 and a == 1 and s_p == s + 1 and R[r] == 0 :
          p[s, a, s_p, r] = 1.0
          
        
        # si on est sur la dernière ligne
        # si on monte
        if 35 < s < 40 and a == 2 and s_p == s - 7 and R[r] == 0 :
          p[s, a, s_p, r] = 1.0
        # si on va à gauche
        if 36 < s < 40 and a == 0 and s_p == s - 1 and R[r] == 0 :
          p[s, a, s_p, r] = 1.0
        # si on va à droite
        if 35 < s < 39 and a == 0 and s_p == s - 1 and R[r] == 0 :
          p[s, a, s_p, r] = 1.0
          

# coup spéciaux : 

p[11][1][12][0] = 1.0
p[19][2][12][0] = 1.0
p[39][1][40][3] = 1.0
p[33][3][40][3] = 1.0

In [3]:
def naive_policy_iteration():
  theta = 0.00001
  gamma = 0.999

  V = np.random.random(len(S))
  for s in T:
    V[s] = 0.0
  Pi = np.random.choice(A, len(S), True)

  while True:
    # Policy evaluation
    while True:
      delta = 0
      for s in range(len(S)):
        v = V[s]
        total = 0
        for s_p in range(len(S)):
          for r in range(len(R)):
            total += p[s, Pi[s], s_p, r] * (R[r] + 0.999 * V[s_p])
        V[s] = total
        delta = np.maximum(delta, np.abs(v - V[s]))
      if delta < theta:
        break

    # Policy improvement
    policy_stable = True

    for s in S:
      if s in T:
        continue

      old_action = Pi[s]

      # Compute Arg Max a
      argmax_a = None
      max_a = -999999999

      for a in A:
        total = 0.0

        for s_p in S:
          for r_index in range(len(R)):
            total += p[s, a, s_p, r_index] * (R[r_index] + gamma * V[s_p])

        if argmax_a is None or total >= max_a:
          argmax_a = a
          max_a = total

      Pi[s] = argmax_a

      if old_action != Pi[s]:
        policy_stable = False


    if policy_stable:
      break

  return Pi

In [6]:
tab = naive_policy_iteration()

In [7]:
print(tab.reshape(7,7))

[[3 3 3 0 1 3 1]
 [3 3 3 3 3 2 2]
 [1 3 3 3 3 3 1]
 [0 3 3 3 3 3 1]
 [1 1 1 1 3 3 0]
 [2 2 2 2 1 2 3]
 [3 1 1 1 2 3 2]]


In [8]:
from colorama import Fore, Style

def print_grid_with_color(grid):
    for row in grid:
        for cell in row:
            if cell == 'X':
                print(Fore.RED + cell, end=' ')
            elif cell == 'I':
                print(Fore.MAGENTA + cell, end=' ')
            elif cell == 'O':
                print(Fore.YELLOW + cell, end=' ')
        print(Style.RESET_ALL)  # Réinitialiser la couleur après chaque ligne
        
# Fonction pour exécuter le jeu
def run_game(Pi, start_row, start_col):
    # Créez la grille de jeu 6x7 remplie de 0
    grid = np.zeros((7, 7), dtype=str)

    # Initialisez les bords de la grille avec 'I'
    grid[:, 0] = 'I'  # première colonne
    grid[:, 6] = 'I'  # dernière colonne
    grid[0, :] = 'I'  # première ligne
    grid[6, :] = 'I'  # dernière ligne
    grid[1:6, 1:6] = 'O' # autres lignes

    # Position de départ
    current_row, current_col = start_row, start_col

    print("Position de départ :\n")
    # Mettez à jour la grille avec le "X" de la nouvelle position
    grid[current_row, current_col] = 'X'

    # Affichez la grille mise à jour
    print_grid_with_color(grid)
    print('\n')
    print("-------------\n")

    # Exécutez le jeu étape par étape
    while True:
        # Supprimez le "X" du départ s'il existe
        grid[current_row, current_col] = 'O'

        print("Position actuelle : ({}, {})\n".format(current_row, current_col))
        action = Pi[current_row * 7 + current_col]  # Convertir les coordonnées en indice unique

        # Mettez à jour la position en fonction de l'action
        if action == 0:  # left
            current_col = max(current_col - 1, 1)
        elif action == 1:  # right
            current_col = min(current_col + 1, 5)
        elif action == 2:  # up
            current_row = max(current_row - 1, 1)
        elif action == 3:  # down
            current_row = min(current_row + 1, 4)

        # Mettez à jour la grille avec le "X" de la nouvelle position
        grid[current_row, current_col] = 'X'

        # Affichez la grille mise à jour
        print_grid_with_color(grid)
        print('\n')
        print('-------------------\n')

        # Si la position actuelle est terminale, arrêtez le jeu
        if (current_row, current_col) in [(1, 5), (5, 5)]:
            print("Position terminale atteinte. Le jeu est terminé.")
            break


In [ ]:
run_game(tab, 1, 1)